In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange

# Load the data
mnist_train = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False)
import torch.nn as nn

class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(7*7*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        # conv layer 1
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
        # conv layer 2
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
        # fc layer 1
        x = x.view(-1, 7*7*64)
        x = self.fc1(x)
        x = F.relu(x)
        
        # fc layer 2
        x = self.fc2(x)
        return x  
## Training
# Instantiate model  
model = MNIST_CNN()  # <---- change here
print(model)
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # <---- change here

# Iterate through train set minibatchs 
for epoch in trange(3):  # <---- change here
    for images, labels in tqdm(train_loader):
        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        x = images  # <---- change here 
        y = model(x)
        loss = criterion(y, labels)
        # Backward pass
        loss.backward()
        optimizer.step()

## Testing
correct = 0
total = len(mnist_test)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_loader):
        # Forward pass
        x = images  # <---- change here 
        y = model(x)

        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())

print('Test accuracy: {}'.format(correct/total))

MNIST_CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=3136, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)



Test accuracy: 0.9901999831199646


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
import numpy as np
from torchvision import datasets,transforms
import torch.optim as optim

trainset = mnist_train
testset = mnist_test

trainloader = torch.utils.data.DataLoader(trainset,batch_size=100,shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=100,shuffle=False)

class CNN_NEW(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,32,kernel_size=3)
    self.conv2 = nn.Conv2d(32,32,kernel_size=3)
    self.conv3 = nn.Conv2d(32,64,kernel_size=3)
    self.conv4 = nn.Conv2d(64,64,kernel_size=3)
    self.fc1 = nn.Linear(20736,256)
    self.fc2 = nn.Linear(256,10)

  def forward(self,X):
    X = F.relu(self.conv1(X))
    X = F.relu(self.conv2(X))
    max_pool = nn.MaxPool2d(2,1)
    X = max_pool(X)
    X = F.relu(self.conv3(X))
    X = F.relu(self.conv4(X))
    X = max_pool(X)
    #print(X.flatten().shape)
    X = self.fc1(X.view(100,20736))
    X = self.fc2(X)
    #print(X.shape)
    #return F.softmax(X,dim=1)
    return X

if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('running on gpu')
else:
  device = torch.device('cpu')
  print('running on cpu')
    
mymodel = CNN_NEW().to(device)

BATCH_SIZE=100
EPOCHS=3

optimizer = optim.Adam(mymodel.parameters(),lr=0.001)
Loss = nn.CrossEntropyLoss()

batches = iter(trainloader)

for e in range(EPOCHS):
  for batch in tqdm(batches):
    mymodel.zero_grad()
    image, label = batch
    image = image.to(device)
    label = label.to(device)
    #print(image.shape)

    Y = mymodel(image)
    #print(Y.shape,label.shape)
    loss = Loss(Y,label)
    loss.backward()
    optimizer.step()
  batches = iter(trainloader)

tests = iter(testloader)
correct = 0
total = 0

with torch.no_grad():
  for batch in tqdm(tests):
    img, lbl = batch
    imag = img.to(device)
    lbl = lbl.to(device)
    Y = mymodel(imag)
    predict = torch.argmax(Y,dim=1)
    
#   plt.imshow(img[1].view(28,28))
#    plt.title(Y)

    for p,l in zip(predict,lbl):
      #print(predict,lbl)
      if p == l:
        correct+=1
      total+=1

print("Accuracy: {}%".format((correct/total)*100))
#print(correct,total)

running on cpu



Accuracy: 98.91%
